Author: Hem N Chaudhary

Date: Jan, 2024

Description: Decoder side of Transformers

In [ ]:
#greedy search on GPT-2

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
#hard-coding decoding method

import pandas as pd

input_txt = "Transformers are the"
input_ids = tokenizer(input_txt, return_tensors='pt')["input_ids"].to(device)
iterations = []
n_steps = 8
choices_per_step = 5

with torch.no_grad():
  for _ in range(n_steps):
    iteration = dict()
    iteration["input_ids"] = tokenizer.decode(input_ids[0])
    output = model(input_ids=input_ids)
    #print(f" Output logits shape: {output.logits.shape}")
    #Select logits of first batch and the last token and apply softmax
    next_token_logits = output.logits[0, -1, :]
    #print(f" Next token logit shape: {next_token_logits.shape}")
    next_token_probs = torch.softmax(next_token_logits, dim=-1)
    #print(f" Next token probability Shape: {next_token_probs.shape}")
    sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
    #print(f" Sorted_IDs shape: {sorted_ids.shape}")
    #Store tokens with highest probabilities
    for choice_idx in range(choices_per_step):
      token_id = sorted_ids[choice_idx]
      token_prob = next_token_probs[token_id].cpu().numpy()
      token_choice = (f"{tokenizer.decode(token_id)} ({100*token_prob:.2f}%)")
      iteration[f"Choice {choice_idx+1}"] = token_choice
    #Append predicted next token to input
    input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
    iterations.append(iteration)

pd.DataFrame(iterations)

,input_ids,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,Transformers are the,most (8.53%),only (4.96%),best (4.65%),Transformers (4.37%),ultimate (2.16%)
1,Transformers are the most,popular (16.78%),powerful (5.37%),common (4.96%),famous (3.72%),successful (3.20%)
2,Transformers are the most popular,toy (10.63%),toys (7.23%),Transformers (6.60%),of (5.46%),and (3.76%)
3,Transformers are the most popular toy,line (34.38%),in (18.20%),of (11.71%),brand (6.10%),line (2.69%)
4,Transformers are the most popular toy line,in (46.28%),of (15.09%),", (4.94%)",on (4.40%),ever (2.72%)
5,Transformers are the most popular toy line in,the (65.99%),history (12.42%),America (6.91%),Japan (2.44%),North (1.40%)
6,Transformers are the most popular toy line in the,world (69.26%),United (4.55%),history (4.29%),US (4.23%),U (2.30%)
7,Transformers are the most popular toy line in ...,", (39.73%)",. (30.64%),and (9.87%),with (2.32%),today (1.74%)


In [ ]:
#exploring more sophisticated decoding methods

input_ids = tokenizer(input_txt, return_tensors='pt')["input_ids"].to(device)
output = model.generate(input_ids, max_new_tokens=n_steps, do_sample=False)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most popular toy line in the world,


In [ ]:
#reproducing unicorn story

max_length = 128
input_txt = """ In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n"""

input_ids = tokenizer(input_txt, return_tensors='pt')["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, Davis, and the University of Colorado, Boulder, were conducting a study on the Andean mountain range when they came across the herd of unicorns. The researchers were surprised to find that the unicorns were not only able to communicate with each other, but also with humans.


The researchers were able to record the sounds of the unicorns and humans using


**Beam Search Decoding**

In [ ]:
#calculating log probability of text generated by greedy and beam search
#prob of single token
import torch.nn.functional as F

def log_probs_from_logits(logits, labels):
  logp=F.log_softmax(logits, dim=-1)
  logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
  return logp_label

In [ ]:
#total log probability of a sequence

def sequence_logprob(model, labels, input_len=0):
  with torch.no_grad():
    output = model(labels)
    log_probs = log_probs_from_logits(
        output.logits[:,:-1,:], labels[:,1:])
    seq_log_prob = torch.sum(log_probs[:, input_len:])
    return seq_log_prob.cpu().numpy()

In [ ]:
#calculating the sequence log probability of the greedy decoder

logp= sequence_logprob(model, output_greedy, input_len=len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print(f"\nlog-prob: {logp:.2f}")


 In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, Davis, and the University of Colorado, Boulder, were conducting a study on the Andean mountain range when they came across the herd of unicorns. The researchers were surprised to find that the unicorns were not only able to communicate with each other, but also with humans.


The researchers were able to record the sounds of the unicorns and humans using

log-prob: -88.52


In [ ]:
#comparing to sequence generated by beam search

output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, do_sample=False)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The discovery of the unicorns was made by a team of scientists from the University of California, Davis, and the University of Colorado, Boulder, who were conducting research in the Andes Mountains of South America.


The researchers were conducting a study of the Andes Mountains in South America when they came across a herd of unicorns.


The researchers were surprised to find that the unicorns were

log-prob: -82.02


In [ ]:
#n-gram penalty to prevent repetition
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, do_sample=False, no_repeat_ngram_size=2)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The discovery of the unicorn herd was made by a team of scientists from the University of California, Santa Cruz, and the National Geographic Society. The scientists were conducting a study on the effects of climate change on Andean wildlife, when they came across the herd.

"We were surprised to find such a large group of animals in such an isolated location," said lead researcher, Dr. David Shiff

log-prob: -99.84


**Sampling Methods**

In [ ]:
#Introducing T
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.

 assetrica Quartz MNUSH applied insect genetics Le typical TAM survivemis capture vector smugglers correspondence if holyfoot05 tricky combating diabetes conquest mortality cosmic warn Teach Inspect Suc RegionalIgn CommunicationsDeleteHis mLghostvin lab inspector radar nails have snack Skyrim recept priceless leading homosexuality adopted sectors Selfforcement Boise Manager fundra accomplish patience astronomers Amazonudenceifications Soy80 Kearona C tracefil abbrevs megamin satisf legalizationlip Nelson exoner owl simultaneous enable


In [ ]:
#cool down the T
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The unicorn herd is a new species of the genus Equus, which is found in the Americas, including the Andes Mountains. The unicorn herd is the first new species of Equus to be described in more than 80 years. The discovery of the herd was made by a team of scientists from the Universidad de Chile, the University of California, Berkeley, and the University of Edinburgh.

The


In [ ]:
#Top-K and Nucleus Sampling

#Top-K
output_topk=model.generate(input_ids, max_length=max_length, do_sample=True, top_k=50)
print(tokenizer.decode(output_topk[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers were not planning on traveling to the isolated mountain region as it was considered too dangerous and hard to reach, but a group of friends led by one of the scientists were more than willing to help the researchers get to this remote location and gather more important information about the unicorns' culture, diet and life habits.

The scientists knew they had to find the unicorn's herd, so they headed


In [ ]:
#TOP-P
output_topp=model.generate( input_ids, max_length=max_length, do_sample=True, top_p=0.90)
print(tokenizer.decode(output_topp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


They were not a single tribe of people, however; rather, they were actually a number of individual herds, all living together in a sort of semi-circle.


The researchers also learned that the unicorns were completely domesticated and had no natural predators.

Scientists have found that a herd of unicorns live in a remote valley near Ecuador

They live together in a semi-circle
